# Classification

Time needed to make a prediction for all 10 joints of hand. Includes the time needed to load the image.

In [1]:
from tensorflow.python.keras.models import load_model
import h5py
clas_model = load_model("/cluster/home/rohrbjan/joint_scoring/models/20171108_195245.h5")

In [2]:
import time
start = time.time()
with h5py.File('/cluster/home/rohrbjan/joint_scoring/img_test.h5', 'r') as hf:
    img_tmp = hf['img_test'][0:10]
tmp = clas_model.predict(img_tmp)
end = time.time()
print(end - start)

1.1243374347686768


# Regression

Time needed to make a prediction for all 10 joints of hand. Includes the time needed to load the image and transform it from greyscale to rgb, as well as the time needed for the transformation to the range [-1, 1]

In [3]:
from PIL import Image
from tensorflow.python.keras.applications.inception_v3 import preprocess_input
import numpy as np

from tensorflow.python.keras import backend as K

def crps(y_true, y_pred):
    diff = (y_true - y_pred)
    diff_2 = K.square(diff)
    loss = K.mean(diff_2)
    return loss

reg_model = load_model("/cluster/home/rohrbjan/joint_scoring/models/20171128_000703.h5", custom_objects={"crps":crps})

In [4]:
start = time.time()
with h5py.File('/cluster/home/rohrbjan/joint_scoring/img_test_regression.h5', 'r') as hf:
    img_tmp = hf['img_test_regression'][0:10]

img_tmp_rgb = []

for i, img in enumerate(img_tmp):
    im = Image.fromarray((img.squeeze()).astype("int16"))
    im = im.convert("RGB")
    dat = np.array(im)
    img_tmp_rgb.append(dat)

img_tmp_rgb = [preprocess_input(x.astype("float32")) for x in img_tmp_rgb]
img_tmp_rgb = np.array(img_tmp_rgb)

tmp = reg_model.predict(img_tmp_rgb)
end = time.time()
print(end - start)

1.0139703750610352
